# **Aspect-based Sentiment Analysis using BERTs**

##**Dataset**

In [ ]:
!pip install -q datasets==3.2.0

In [2]:
from datasets import load_dataset

ds = load_dataset("thainq107/abte-restaurants")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3602 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Tags', 'Polarities'],
        num_rows: 3602
    })
    test: Dataset({
        features: ['Tokens', 'Tags', 'Polarities'],
        num_rows: 1119
    })
})

## **Tokenizer**

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") # "albert-base-v2"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tokenize_and_align_labels(examples):
    sentences, sentence_tags = [], []
    labels = []
    for tokens, pols in zip(examples['Tokens'], examples['Polarities']):

        bert_tokens = []
        bert_att = []
        pols_label = 0
        for i in range(len(tokens)):
            t = tokenizer.tokenize(tokens[i])
            bert_tokens += t
            if int(pols[i]) != -1:
                bert_att += t
                pols_label = int(pols[i])

        sentences.append(" ".join(bert_tokens))
        sentence_tags.append(" ".join(bert_att))
        labels.append(pols_label)

    tokenized_inputs = tokenizer(sentences, sentence_tags, padding=True, truncation=True, return_tensors="pt")
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [6]:
preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [7]:
preprocessed_ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3602
    })
    test: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1119
    })
})

## **Evaluate**

In [8]:
!pip install -q evaluate==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

##**Model**

In [11]:
from transformers import AutoModelForSequenceClassification

id2label = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
label2id = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=3, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## **Training**

In [13]:
# Disable wandb
import os
os.environ['WANDB_DISABLED'] = 'true'

# # Use wandb
# import wandb
# wandb.init(
#     project="aspect-based-sentiment-analysis",
#     name="distilbert/distilbert-base-uncased" # "transformer-encoder", "lstm", "conv1d"
# )

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="absa-restaurants-albert-base-v2",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=50,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.938200,0.747046,0.649687
2,0.694000,0.572286,0.765862
3,0.578200,0.518178,0.787310
4,0.489600,0.512656,0.800715
5,0.410200,0.473091,0.810545
6,0.356500,0.465567,0.823950
7,0.289800,0.521200,0.812332
8,0.255000,0.494295,0.831993
9,0.209500,0.574544,0.816801
10,0.178500,0.571794,0.823056


TrainOutput(global_step=1450, training_loss=0.11577122234064957, metrics={'train_runtime': 2734.1458, 'train_samples_per_second': 65.871, 'train_steps_per_second': 0.53, 'total_flos': 6896396457309600.0, 'train_loss': 0.11577122234064957, 'epoch': 50.0})

## **Inference**

In [16]:
trainer.save_model("abte-restaurants-distilbert-base-uncased")
tokenizer.save_pretrained("abte-restaurants-distilbert-base-uncased")

('abte-restaurants-distilbert-base-uncased/tokenizer_config.json',
 'abte-restaurants-distilbert-base-uncased/special_tokens_map.json',
 'abte-restaurants-distilbert-base-uncased/vocab.txt',
 'abte-restaurants-distilbert-base-uncased/added_tokens.json',
 'abte-restaurants-distilbert-base-uncased/tokenizer.json')

In [17]:
from transformers import pipeline

token_classifier = pipeline(
    "ner",
    model="abte-restaurants-distilbert-base-uncased",
    tokenizer="abte-restaurants-distilbert-base-uncased",
    aggregation_strategy="simple"  # This returns individual tokens
)

classifier = pipeline(
    "text-classification",
    model="abte-restaurants-distilbert-base-uncased",
    tokenizer="abte-restaurants-distilbert-base-uncased",
)


test_sentence = "The bread is top notch as well"
results = token_classifier(test_sentence)

# Joining recognized words from the NER pipeline
sentence_tags = " ".join([result["word"] for result in results])

# Classifying the sentence with additional sentence tags
pred_label = classifier(f"{test_sentence} [SEP] {sentence_tags}")

sentence_tags, pred_label

Device set to use cuda:0
Device set to use cuda:0


('the bread is top notch as well',
 [{'label': 'Positive', 'score': 0.987366795539856}])